# 01 - Data Gathering

## Description
This notebook gathers the necissary data from 3 different data sources:
 - 2022Q3 Divvy bike/trip data found on [Divvy's Data Portal](https://divvy-tripdata.s3.amazonaws.com/index.html)
 - Estimated travel time using  [Open Source Routing Machine (OSRM) API](http://project-osrm.org/docs/v5.10.0/api/#general-options)
 - Historic weather data from [OpenWeather API](https://openweathermap.org/api/one-call-3#data)

## Step 0: Setup Libraries and File Locations

### Import Libraries

In [7]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests
import zipfile
import io
import datetime
import json
from tqdm import tqdm
from scrapy.selector import Selector
import numpy as np
import config

### Create file location

In [2]:
ROOT = os.getcwd()
SAVE_FILES = os.path.join(ROOT, "DATA")
SAVE_FILES

'C:\\Users\\Nicholas\\Desktop\\Masters - Classes\\MSDS436\\Final\\MSDS436-FINAL\\DATA'

## Step 1: Pull the September Divvy Bike Data

In [3]:
r = requests.get(f"https://divvy-tripdata.s3.amazonaws.com/202209-divvy-tripdata.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(SAVE_FILES)

In [4]:
sept_df = pd.read_csv('C:\\Users\\Nicholas\\Desktop\\Masters - Classes\\MSDS436\\Final\\MSDS436-FINAL\\DATA\\202209-divvy-publictripdata.csv')
sept_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5156990AC19CA285,electric_bike,2022-09-01 08:36:22,2022-09-01 08:39:05,NaN,NaN,California Ave & Milwaukee Ave,13084,41.93,-87.69,41.922695,-87.697153,casual
1,E12D4A16BF51C274,electric_bike,2022-09-01 17:11:29,2022-09-01 17:14:45,NaN,NaN,NaN,NaN,41.87,-87.62,41.870000,-87.620000,casual
2,A02B53CD7DB72DD7,electric_bike,2022-09-01 17:15:50,2022-09-01 17:16:12,NaN,NaN,NaN,NaN,41.87,-87.62,41.870000,-87.620000,casual
3,C82E05FEE872DF11,electric_bike,2022-09-01 09:00:28,2022-09-01 09:10:32,NaN,NaN,NaN,NaN,41.93,-87.69,41.940000,-87.670000,casual
4,4DEEB4550A266AE1,electric_bike,2022-09-01 07:30:11,2022-09-01 07:32:36,NaN,NaN,NaN,NaN,41.92,-87.73,41.920000,-87.730000,casual


## Step 2: Clean data
- Order start_at in decending order
- Drop all rows that do not have a start and end station
- Pull 5,000 data points per day
- Pull weather data for 24h per day (720 rows total)

### Order start_at in decending order

In [5]:
sept_df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [6]:
sept_df['started_at'] = pd.to_datetime(sept_df['started_at'])
sept_df.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at                      object
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

In [7]:
sept_df = sept_df.sort_values(by='started_at')

display(len(sept_df))
sept_df.head()

701339

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
474077,A4BBE90F834C0422,classic_bike,2022-09-01 00:00:05,2022-09-01 01:07:18,Lincoln Ave & Sunnyside Ave,TA1307000156,Manor Ave & Leland Ave,KA1504000127,41.963004,-87.684781,41.965875,-87.700825,member
520136,51DC98F92D41C0CD,electric_bike,2022-09-01 00:00:30,2022-09-01 00:18:54,Clark St & Lincoln Ave,13179,Lincoln Ave & Belle Plaine Ave,TA1309000026,41.915737,-87.634587,41.956004,-87.680161,casual
458774,5349D00EEA419440,electric_bike,2022-09-01 00:01:05,2022-09-01 00:05:22,Ashland Ave & Division St,13061,NaN,NaN,41.903223,-87.668033,41.920000,-87.680000,member
374156,CD46A4DC22FC5B22,classic_bike,2022-09-01 00:01:08,2022-09-01 00:14:16,Loomis St & Lexington St,13332,Morgan St & Polk St,TA1307000130,41.872187,-87.661501,41.871737,-87.651030,member
607263,16A28B1E5249DADB,electric_bike,2022-09-01 00:01:14,2022-09-01 00:14:25,Malcolm X College Vaccination Site,631,Kedzie Ave & Roosevelt Rd,15682,41.877639,-87.673811,41.866493,-87.706496,casual


### Drop all rows that do not have a start and end station name

In [8]:
sept_df = sept_df[sept_df['start_station_name'].notnull()]
sept_df = sept_df[sept_df['end_station_name'].notnull()]

display(len(sept_df))
sept_df.head()

535145

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
474077,A4BBE90F834C0422,classic_bike,2022-09-01 00:00:05,2022-09-01 01:07:18,Lincoln Ave & Sunnyside Ave,TA1307000156,Manor Ave & Leland Ave,KA1504000127,41.963004,-87.684781,41.965875,-87.700825,member
520136,51DC98F92D41C0CD,electric_bike,2022-09-01 00:00:30,2022-09-01 00:18:54,Clark St & Lincoln Ave,13179,Lincoln Ave & Belle Plaine Ave,TA1309000026,41.915737,-87.634587,41.956004,-87.680161,casual
374156,CD46A4DC22FC5B22,classic_bike,2022-09-01 00:01:08,2022-09-01 00:14:16,Loomis St & Lexington St,13332,Morgan St & Polk St,TA1307000130,41.872187,-87.661501,41.871737,-87.651030,member
607263,16A28B1E5249DADB,electric_bike,2022-09-01 00:01:14,2022-09-01 00:14:25,Malcolm X College Vaccination Site,631,Kedzie Ave & Roosevelt Rd,15682,41.877639,-87.673811,41.866493,-87.706496,casual
632892,BD4CB89D2A406B3D,classic_bike,2022-09-01 00:01:50,2022-09-01 00:15:04,Lincoln Ave & Fullerton Ave,TA1309000058,Ashland Ave & Wellington Ave,13269,41.924161,-87.646380,41.936083,-87.669807,member


### Grab up to 5,000 rows per day

In [9]:
# get just date column
sept_df['started_at_clean'] = sept_df['started_at'].dt.date.astype(str)
sept_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_at_clean
474077,A4BBE90F834C0422,classic_bike,2022-09-01 00:00:05,2022-09-01 01:07:18,Lincoln Ave & Sunnyside Ave,TA1307000156,Manor Ave & Leland Ave,KA1504000127,41.963004,-87.684781,41.965875,-87.700825,member,2022-09-01
520136,51DC98F92D41C0CD,electric_bike,2022-09-01 00:00:30,2022-09-01 00:18:54,Clark St & Lincoln Ave,13179,Lincoln Ave & Belle Plaine Ave,TA1309000026,41.915737,-87.634587,41.956004,-87.680161,casual,2022-09-01
374156,CD46A4DC22FC5B22,classic_bike,2022-09-01 00:01:08,2022-09-01 00:14:16,Loomis St & Lexington St,13332,Morgan St & Polk St,TA1307000130,41.872187,-87.661501,41.871737,-87.651030,member,2022-09-01
607263,16A28B1E5249DADB,electric_bike,2022-09-01 00:01:14,2022-09-01 00:14:25,Malcolm X College Vaccination Site,631,Kedzie Ave & Roosevelt Rd,15682,41.877639,-87.673811,41.866493,-87.706496,casual,2022-09-01
632892,BD4CB89D2A406B3D,classic_bike,2022-09-01 00:01:50,2022-09-01 00:15:04,Lincoln Ave & Fullerton Ave,TA1309000058,Ashland Ave & Wellington Ave,13269,41.924161,-87.646380,41.936083,-87.669807,member,2022-09-01


In [10]:
# create unique date list
date_ls = sept_df['started_at_clean'].unique().tolist()
date_ls[:5]

['2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05']

In [12]:
cnt = 0
for date in date_ls:
    if cnt == 0:
        main_df = sept_df[sept_df['started_at_clean'] == date].sample(n=5000, random_state=0)
        cnt =+ 1
    else:
        filter_df = sept_df[sept_df['started_at_clean'] == date].sample(n=5000, random_state=0)
        main_df = pd.concat([main_df, filter_df], ignore_index=True, axis=0)

In [13]:
display(len(main_df))

main_df.index.name = 'row'
main_df.head()

150000

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_at_clean
row,,,,,,,,,,,,,,
0,2FD3F90EDCE2ACD9,classic_bike,2022-09-01 19:39:15,2022-09-01 19:46:45,Southport Ave & Roscoe St,13071,Broadway & Cornelia Ave,13278,41.943739,-87.664020,41.945529,-87.646439,member,2022-09-01
1,EE62794A94F80A83,classic_bike,2022-09-01 06:53:41,2022-09-01 07:02:54,LaSalle St & Washington St,13006,Wells St & Polk St,SL-011,41.882664,-87.632530,41.872732,-87.633516,casual,2022-09-01
2,56FD4B364747F270,electric_bike,2022-09-01 11:25:21,2022-09-01 11:28:33,N Sheffield Ave & W Wellington Ave,20256.0,Southport Ave & Wellington Ave,TA1307000006,41.940000,-87.650000,41.935733,-87.663576,casual,2022-09-01
3,BD4D6AC842CDF729,classic_bike,2022-09-01 07:46:03,2022-09-01 08:05:36,Racine Ave & Wrightwood Ave,TA1309000059,DuSable Lake Shore Dr & North Blvd,LF-005,41.928887,-87.658971,41.911722,-87.626804,casual,2022-09-01
4,2E0E8C378865C01A,electric_bike,2022-09-01 09:55:31,2022-09-01 10:12:27,Wabash Ave & Adams St,KA1503000015,Wood St & Taylor St (Temp),13285,41.879373,-87.625492,41.869265,-87.673731,member,2022-09-01


In [ ]:
# # check data
# main_df.to_csv("data_check.csv", index=Flase)

## Step 3: Get estimated travel time

In [14]:
# new method per https://github.com/Project-OSRM/osrm-backend/issues/6258
def get_distance_bike(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    
    url = f"""https://routing.openstreetmap.de/routed-bike/route/v1/biking/{point1["start_lng"]},{point1["start_lat"]};{point2["end_lng"]},{point2["end_lat"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

In [15]:
# get the distances and durations
dist_array_bike = []
for i , r in tqdm(main_df.iterrows()):
    try:
        point1 = {"start_lat": r["start_lat"], "start_lng": r["start_lng"]}
        point2 = {"end_lat": r["end_lat"], "end_lng": r["end_lng"]}
        dist, duration = get_distance_bike(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        dist_array_bike.append((i, duration, dist))
    except KeyError:
        dist_array_bike.append((i, 0, 0))
        continue

150000it [17:33:41,  2.37it/s]


In [16]:
distances_bike_df = pd.DataFrame(dist_array_bike,columns=["row","duration (s)","distance (m)"])

display(len(distances_bike_df))
distances_bike_df.head()

150000

,row,duration (s),distance (m)
0,0,481.9,1669.2
1,1,395.0,1358.8
2,2,411.5,1595.7
3,3,948.8,3643.8
4,4,1275.1,5104.7


In [17]:
# Matches key value for 'row' or any other unique identifier we want to assign later on
sep_dis_df = pd.merge(main_df, distances_bike_df, on='row', how='right').drop('row', axis=1)
sep_dis_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_at_clean,duration (s),distance (m)
0,2FD3F90EDCE2ACD9,classic_bike,2022-09-01 19:39:15,2022-09-01 19:46:45,Southport Ave & Roscoe St,13071,Broadway & Cornelia Ave,13278,41.943739,-87.664020,41.945529,-87.646439,member,2022-09-01,481.9,1669.2
1,EE62794A94F80A83,classic_bike,2022-09-01 06:53:41,2022-09-01 07:02:54,LaSalle St & Washington St,13006,Wells St & Polk St,SL-011,41.882664,-87.632530,41.872732,-87.633516,casual,2022-09-01,395.0,1358.8
2,56FD4B364747F270,electric_bike,2022-09-01 11:25:21,2022-09-01 11:28:33,N Sheffield Ave & W Wellington Ave,20256.0,Southport Ave & Wellington Ave,TA1307000006,41.940000,-87.650000,41.935733,-87.663576,casual,2022-09-01,411.5,1595.7
3,BD4D6AC842CDF729,classic_bike,2022-09-01 07:46:03,2022-09-01 08:05:36,Racine Ave & Wrightwood Ave,TA1309000059,DuSable Lake Shore Dr & North Blvd,LF-005,41.928887,-87.658971,41.911722,-87.626804,casual,2022-09-01,948.8,3643.8
4,2E0E8C378865C01A,electric_bike,2022-09-01 09:55:31,2022-09-01 10:12:27,Wabash Ave & Adams St,KA1503000015,Wood St & Taylor St (Temp),13285,41.879373,-87.625492,41.869265,-87.673731,member,2022-09-01,1275.1,5104.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,47FF8C0117865527,electric_bike,2022-09-30 23:35:19,2022-09-30 23:46:31,Kedzie Ave & Milwaukee Ave,13085,California Ave & Cortez St,17660,41.929660,-87.708040,41.900363,-87.696704,casual,2022-09-30,926.6,3719.3
149996,55CAF601C880187D,electric_bike,2022-09-30 19:04:29,2022-09-30 19:16:23,Francisco Ave & Montrose Ave,446,Hoyne Ave & Balmoral Ave,655,41.960000,-87.700000,41.979851,-87.681932,casual,2022-09-30,914.3,3559.1
149997,DA0C6AF5AD6D59E1,classic_bike,2022-09-30 07:34:53,2022-09-30 07:48:19,Bissell St & Armitage Ave*,chargingstx1,Clark St & North Ave,13128,41.918296,-87.652183,41.911974,-87.631942,casual,2022-09-30,554.2,2105.2
149998,E574F285BB055A8E,classic_bike,2022-09-30 13:05:43,2022-09-30 13:11:23,Clark St & Wrightwood Ave,TA1305000014,Clark St & Armitage Ave,13146,41.929546,-87.643118,41.918306,-87.636282,member,2022-09-30,353.1,1411.4


In [18]:
# Save data frame
sep_dis_df.to_csv("DATA/202209_divvy_distance.csv", index=False)

## Step 4: Collect historic weather data
- Chicago lat: 41.87
- Chicago long: -87.62
- Pull hourly data for all 30 days (720 rows)

In [19]:
# Load save file
sep_dis_df = pd.read_csv("DATA/202209_divvy_distance.csv")

In [20]:
sep_dis_df['started_at_unix'] = pd.to_datetime(sep_dis_df['started_at'])
sep_dis_df['started_at_unix'] = pd.to_datetime(sep_dis_df['started_at_unix'].dt.strftime('%Y-%m-%d %H'))
sep_dis_df['started_at_unix'] = pd.to_numeric(sep_dis_df['started_at_unix'])
sep_dis_df['started_at_unix'] = sep_dis_df['started_at_unix'] // 10 ** 9


sep_dis_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_at_clean,duration (s),distance (m),started_at_unix
0,2FD3F90EDCE2ACD9,classic_bike,2022-09-01 19:39:15,2022-09-01 19:46:45,Southport Ave & Roscoe St,13071,Broadway & Cornelia Ave,13278,41.943739,-87.664020,41.945529,-87.646439,member,2022-09-01,481.9,1669.2,1662058800
1,EE62794A94F80A83,classic_bike,2022-09-01 06:53:41,2022-09-01 07:02:54,LaSalle St & Washington St,13006,Wells St & Polk St,SL-011,41.882664,-87.632530,41.872732,-87.633516,casual,2022-09-01,395.0,1358.8,1662012000
2,56FD4B364747F270,electric_bike,2022-09-01 11:25:21,2022-09-01 11:28:33,N Sheffield Ave & W Wellington Ave,20256.0,Southport Ave & Wellington Ave,TA1307000006,41.940000,-87.650000,41.935733,-87.663576,casual,2022-09-01,411.5,1595.7,1662030000
3,BD4D6AC842CDF729,classic_bike,2022-09-01 07:46:03,2022-09-01 08:05:36,Racine Ave & Wrightwood Ave,TA1309000059,DuSable Lake Shore Dr & North Blvd,LF-005,41.928887,-87.658971,41.911722,-87.626804,casual,2022-09-01,948.8,3643.8,1662015600
4,2E0E8C378865C01A,electric_bike,2022-09-01 09:55:31,2022-09-01 10:12:27,Wabash Ave & Adams St,KA1503000015,Wood St & Taylor St (Temp),13285,41.879373,-87.625492,41.869265,-87.673731,member,2022-09-01,1275.1,5104.7,1662022800


In [21]:
# Get unique list of unix times
unix_dt_ls = sep_dis_df['started_at_unix'].unique().tolist()
display(unix_dt_ls[:5])
display(len(unix_dt_ls))

[1662058800, 1662012000, 1662030000, 1662015600, 1662022800]

720

In [22]:
def weathermap(latnum, lngnum, dtnum):
    '''
    Pulls weather data using lat, long, and unix_dt
    '''
    api_url = f"http://api.openweathermap.org/data/3.0/onecall/timemachine?lat={latnum}&lon={lngnum}&dt={dtnum}&units=imperial&appid={config.api_key}"
    response = requests.get(api_url)
    resp = response.json()
    
    temp = resp['data'][0]['temp']
    hum = resp['data'][0]['humidity']
    windsp = resp['data'][0]['wind_speed']
    weather = resp['data'][0]['weather'][0]['main']
    try:
        rain = resp['data'][0]['rain']['1h']
    except KeyError as ke:
        rain = 0    
    try:
        snow = resp['data'][0]['snow']['1h']
    except KeyError as ke:
        snow = 0
    
    return temp, hum, windsp, weather, rain, snow

In [23]:
temp_ls = []
hum_ls = []
windsp_ls = []
weather_ls = []
rain_ls = []
snow_ls = []


for unix in tqdm(unix_dt_ls):
    try:
        latnum = 41.87
        lngnum = -87.62
        dtnum = unix
        data = weathermap(latnum, lngnum, dtnum)
        temp_ls.append(data[0])
        hum_ls.append(data[1])
        windsp_ls.append(data[2])
        weather_ls.append(data[3])
        rain_ls.append(data[4])
        snow_ls.append(data[5])
    except NameError:
        temp_ls.append('Nan')
        hum_ls.append('Nan')
        windsp_ls.append('Nan')
        weather_ls.append('Nan')
        rain_ls.append('Nan')
        snow_ls.append('Nan')

100%|████████████████████████████████████████| 720/720 [04:37<00:00,  2.60it/s]


In [7]:
# # API Check

# latnum = 41.87
# lngnum = -87.62

# api_url = f"http://api.openweathermap.org/data/3.0/onecall/timemachine?lat={latnum}&lon={lngnum}&dt=1664568000&units=imperial&appid={config.api_key}"
# response = requests.get(api_url)
# resp = response.json()

# resp

{'lat': 41.87,
 'lon': -87.62,
 'timezone': 'America/Chicago',
 'timezone_offset': -18000,
 'data': [{'dt': 1664568000,
   'sunrise': 1664538373,
   'sunset': 1664580873,
   'temp': 61.39,
   'feels_like': 59.86,
   'pressure': 1024,
   'humidity': 56,
   'dew_point': 45.57,
   'clouds': 0,
   'visibility': 10000,
   'wind_speed': 10.36,
   'wind_deg': 90,
   'wind_gust': 18.41,
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}]}]}

In [24]:
weather_df = pd.DataFrame(list(zip(unix_dt_ls, temp_ls, hum_ls, windsp_ls, 
                                weather_ls, rain_ls, snow_ls)),
               columns =['started_at_unix', 'temp', 'hum', 'windsp', 
                         'weather', 'rain', 'snow'])

weather_df.head()

,started_at_unix,temp,hum,windsp,weather,rain,snow
0,1662058800,89.89,38,8.05,Clouds,0.0,0
1,1662012000,73.96,71,6.91,Clear,0.0,0
2,1662030000,69.58,78,5.75,Clear,0.0,0
3,1662015600,73.54,72,6.91,Clear,0.0,0
4,1662022800,70.81,74,6.91,Clear,0.0,0


In [25]:
## Data check
weather_df[weather_df['started_at_unix']== 1662012000]

,started_at_unix,temp,hum,windsp,weather,rain,snow
1,1662012000,73.96,71,6.91,Clear,0.0,0


In [26]:
# Join data frames on started_at_unix
sep_dis_weath_df = sep_dis_df.merge(weather_df, on='started_at_unix', how='left')
sep_dis_weath_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,started_at_clean,duration (s),distance (m),started_at_unix,temp,hum,windsp,weather,rain,snow
0,2FD3F90EDCE2ACD9,classic_bike,2022-09-01 19:39:15,2022-09-01 19:46:45,Southport Ave & Roscoe St,13071,Broadway & Cornelia Ave,13278,41.943739,-87.664020,...,2022-09-01,481.9,1669.2,1662058800,89.89,38,8.05,Clouds,0.0,0
1,EE62794A94F80A83,classic_bike,2022-09-01 06:53:41,2022-09-01 07:02:54,LaSalle St & Washington St,13006,Wells St & Polk St,SL-011,41.882664,-87.632530,...,2022-09-01,395.0,1358.8,1662012000,73.96,71,6.91,Clear,0.0,0
2,56FD4B364747F270,electric_bike,2022-09-01 11:25:21,2022-09-01 11:28:33,N Sheffield Ave & W Wellington Ave,20256.0,Southport Ave & Wellington Ave,TA1307000006,41.940000,-87.650000,...,2022-09-01,411.5,1595.7,1662030000,69.58,78,5.75,Clear,0.0,0
3,BD4D6AC842CDF729,classic_bike,2022-09-01 07:46:03,2022-09-01 08:05:36,Racine Ave & Wrightwood Ave,TA1309000059,DuSable Lake Shore Dr & North Blvd,LF-005,41.928887,-87.658971,...,2022-09-01,948.8,3643.8,1662015600,73.54,72,6.91,Clear,0.0,0
4,2E0E8C378865C01A,electric_bike,2022-09-01 09:55:31,2022-09-01 10:12:27,Wabash Ave & Adams St,KA1503000015,Wood St & Taylor St (Temp),13285,41.879373,-87.625492,...,2022-09-01,1275.1,5104.7,1662022800,70.81,74,6.91,Clear,0.0,0


In [27]:
# export final data
sep_dis_weath_df.to_csv("DATA/202209_divvy_distance_weather.csv", index=False)

## Step 5: Load CSV file into AWS S3 bucket

In [1]:
import boto3
import io
import os
import config

In [10]:
client = boto3.client(
    's3',
    aws_access_key_id = config.aws_key_id,
    aws_secret_access_key = config.aws_key,
    region_name = 'us-east-2'
)

In [11]:
# Fetch the list of existing buckets
clientResponse = client.list_buckets()
    
# Print the bucket names one by one
print('Printing bucket names...')
for bucket in clientResponse['Buckets']:
    print(f'Bucket Name: {bucket["Name"]}')

Printing bucket names...
Bucket Name: msds436-final


In [14]:
#sep_dis_weath_df = pd.read_csv("DATA/202209_divvy_distance_weather.csv")

In [15]:
# Load the 202209_divvy_distance_weather.csv into S3

with io.StringIO() as csv_buffer:
    sep_dis_weath_df.to_csv(csv_buffer, index=False)
    response = client.put_object(
        Bucket=bucket["Name"], Key="files/202209_divvy_distance_weather.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


## Step 6: Query Postgres DL

In [2]:
import psycopg2

In [5]:
conn =  psycopg2.connect(database="bikeDB",
                         user=config.db_username,
                         password=config.db_password,
                         host="rds-postgresql-final.cxcpgqrdgi5d.us-east-2.rds.amazonaws.com",
                         port='5432'
)

In [8]:
data_df1 = pd.read_sql('''SELECT *
                            FROM "public"."bike_data"
                            Limit 5;
                        ''', conn)
data_df1

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,started_at_clean,duration,distance,started_at_unix,temp,hum,windsp,weather,rain,snow
0,2FD3F90EDCE2ACD9,classic_bike,2022-09-01,2022-09-01,Southport Ave & Roscoe St,13071,Broadway & Cornelia Ave,13278,41.943739,-87.664020,...,2022-09-01,481.9,1669.2,1662058800,89.89,38.0,8.05,Clouds,0.0,0.0
1,EE62794A94F80A83,classic_bike,2022-09-01,2022-09-01,LaSalle St & Washington St,13006,Wells St & Polk St,SL-011,41.882664,-87.632530,...,2022-09-01,395.0,1358.8,1662012000,73.96,71.0,6.91,Clear,0.0,0.0
2,56FD4B364747F270,electric_bike,2022-09-01,2022-09-01,N Sheffield Ave & W Wellington Ave,20256,Southport Ave & Wellington Ave,TA1307000006,41.940000,-87.650000,...,2022-09-01,411.5,1595.7,1662030000,69.58,78.0,5.75,Clear,0.0,0.0
3,BD4D6AC842CDF729,classic_bike,2022-09-01,2022-09-01,Racine Ave & Wrightwood Ave,TA1309000059,DuSable Lake Shore Dr & North Blvd,LF-005,41.928887,-87.658971,...,2022-09-01,948.8,3643.8,1662015600,73.54,72.0,6.91,Clear,0.0,0.0
4,2E0E8C378865C01A,electric_bike,2022-09-01,2022-09-01,Wabash Ave & Adams St,KA1503000015,Wood St & Taylor St (Temp),13285,41.879373,-87.625492,...,2022-09-01,1275.1,5104.7,1662022800,70.81,74.0,6.91,Clear,0.0,0.0


# Full Script
If we had more time, we would use the below code to pull a years worth of bike data and loop through the APIs.

## Step 1: Pull Divvy Bike Data

### Pull keys from website using BeautifulSoup

In [ ]:
main_url = 'https://divvy-tripdata.s3.amazonaws.com'
page = requests.get(main_url)
soup = BeautifulSoup(page.content, 'html.parser')

# print(soup.prettify())

In [ ]:
zip_keys = soup.findAll('key')
len(zip_keys)

### Loop through Keys and only keep divvy-tripdata

In [ ]:
key_ls = []

for i in tqdm(range(len(zip_keys))):
    key_ls.append(zip_keys[i].text)

key_ls_clean = [ x for x in key_ls if "divvy-tripdata" in x ]

key_ls_clean[27:30]

### Pull and save all files

In [ ]:
for zip_f in key_ls_clean[27:30]:
    r = requests.get(f"https://divvy-tripdata.s3.amazonaws.com/{zip_f}")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(SAVE_FILES)

### For our data purpose, read in the 2022Q3 files and create 1 master file
- '202207-divvy-tripdata.csv',
- '202208-divvy-tripdata.csv',
- '202209-divvy-tripdata.csv'

In [ ]:
# Get 202207, 202208, and 202209 files and save file path in list
file_ls = []

for file in os.listdir(SAVE_FILES):
    if file.endswith(".csv"):
        file_ls.append(os.path.join(SAVE_FILES, file))
    
file_ls

In [ ]:
# Loop through files, create pandas data frame, and save in dictonary
df_dict = {}

for i in file_ls:
    for num in range(len(file_ls)):
        df = pd.read_csv(i)
        df_dict[f"df_{num}"] = df

In [ ]:
# Grab dictonary keys and check dataframe
dict_keys_ls = list(df_dict.keys())
df_dict[dict_keys_ls[2]].head()

In [ ]:
sep_df = df_dict[dict_keys_ls[2]]
sep_df.index.name = 'row'
len(sep_df)

In [ ]:
sep_df[:10000].tail()

In [ ]:
# # Concat all 3 data frames and generate Q3_df
# Q3_df = pd.concat([df_dict[dict_keys_ls[0]], df_dict[dict_keys_ls[1]], df_dict[dict_keys_ls[2]]], ignore_index=True, axis=0)
# display(len(Q3_df))
# display(Q3_df.head())

### Get random sample from data

In [ ]:
df_sample = df.sample(n=883, random_state=0)
display(len(df_sample))
df_sample.head()

## Step 2: Get estimated travel time

In [ ]:
#new method per https://github.com/Project-OSRM/osrm-backend/issues/6258
def get_distance_bike(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    
    url = f"""https://routing.openstreetmap.de/routed-bike/route/v1/biking/{point1["start_lng"]},{point1["start_lat"]};{point2["end_lng"]},{point2["end_lat"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

In [ ]:
# get the distances and durations
dist_array_bike = []
for i , r in tqdm(sep_df.iterrows()):
    try:
        point1 = {"start_lat": r["start_lat"], "start_lng": r["start_lng"]}
        point2 = {"end_lat": r["end_lat"], "end_lng": r["end_lng"]}
        dist, duration = get_distance_bike(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        dist_array_bike.append((i, duration, dist))
    except KeyError:
        dist_array_bike.append((i, 0, 0))
        continue

In [ ]:
dist_array_bike[-1]

In [ ]:
# ensure values are different
# print(dist_array_car) ---> [(0, 800.9, 3224.7), (1, 1289.7, 4141.1)]
print(len(dist_array_bike))

In [ ]:
distances_bike_df = pd.DataFrame(dist_array_bike,columns=["row","duration (s)","distance (m)"])
distances_bike_df.head()

In [ ]:
# Matches key value for 'row' or any other unique identifier we want to assign later on
sep_dis_df = pd.merge(sep_df.iloc[:290673], distances_bike_df, on='row', how='right').drop('row', axis=1)
sep_dis_df

In [ ]:
# Save data frame
sep_dis_df.to_csv("202209_divvy_distance.csv")

In [ ]:
# export master file
# Q3_df.to_csv("2022Q3_divvy-tripdata.csv")

## Step 3: Collect historic weather data
**NOTE:** the config.py file contains api_key

### Clean data for weatehr API

In [ ]:
# round lat and long to 2 decimal places (needed for API)
sep_dis_df['start_lat_clean'] = sep_dis_df['start_lat'].round(2)
sep_dis_df['start_lng_clean'] = sep_dis_df['start_lng'].round(2)

sep_dis_df.head()

In [ ]:
sep_dis_df.info()

In [ ]:
sep_dis_df['unix_dt'] = pd.to_datetime(sep_dis_df['started_at'])
sep_dis_df['unix_dt'] = pd.to_datetime(sep_dis_df['unix_dt'])
sep_dis_df['unix_dt'] = pd.to_numeric(sep_dis_df['unix_dt'])
sep_dis_df['unix_dt'] = sep_dis_df['unix_dt'] // 10 ** 9

sep_dis_df.head()

In [ ]:
def weathermap(latnum, lngnum, dtnum):
    '''
    Pulls weather data using lat, long, and unix_dt
    '''
    api_url = f"http://api.openweathermap.org/data/3.0/onecall/timemachine?lat={latnum}&lon={lngnum}&dt={dtnum}&units=imperial&appid={config.api_key}"
    response = requests.get(api_url)
    resp = response.json()
    
    temp = resp['data'][0]['temp']
    hum = resp['data'][0]['humidity']
    windsp = resp['data'][0]['wind_speed']
    weather = resp['data'][0]['weather'][0]['main']
    try:
        rain = resp['data'][0]['rain']['1h']
    except KeyError as ke:
        rain = 0    
    try:
        snow = resp['data'][0]['snow']['1h']
    except KeyError as ke:
        snow = 0
    
    return temp, hum, windsp, weather, rain, snow

In [ ]:
temp_ls = []
hum_ls = []
windsp_ls = []
weather_ls = []
rain_ls = []
snow_ls = []


for i in tqdm(range(len(sep_dis_df))):
    try:
        latnum = sep_dis_df['start_lat'].iloc[i]
        lngnum = sep_dis_df['start_lng'].iloc[i]
        dtnum = sep_dis_df['unix_dt'].iloc[i]
        temp_ls.append(weathermap(latnum, lngnum, dtnum)[0])
        hum_ls.append(weathermap(latnum, lngnum, dtnum)[1])
        windsp_ls.append(weathermap(latnum, lngnum, dtnum)[2])
        weather_ls.append(weathermap(latnum, lngnum, dtnum)[3])
        rain_ls.append(weathermap(latnum, lngnum, dtnum)[4])
        snow_ls.append(weathermap(latnum, lngnum, dtnum)[5])
    except NameError:
        temp_ls.append('Nan')
        hum_ls.append('Nan')
        windsp_ls.append('Nan')
        weather_ls.append('Nan')
        rain_ls.append('Nan')
        snow_ls.append('Nan')
    
sep_dis_df['temp'] = temp_ls
sep_dis_df['hum'] = hum_ls
sep_dis_df['windsp'] = windsp_ls
sep_dis_df['weather'] = weather_ls
sep_dis_df['rain'] = rain_ls
sep_dis_df['snow'] = snow_ls

In [ ]:
len(temp_ls)

## Join historic weather data to sample

In [ ]:
df['temp'] = temp_ls
df['hum'] = hum_ls
df['windsp'] = windsp_ls
df['weather'] = weather_ls
df['rain'] = rain_ls
df['snow'] = snow_ls

### Save data in AWS S3 Bucket

In [ ]:
latnum = sep_dis_df['start_lat'].iloc[0]
latnum

In [ ]:
lngnum = sep_dis_df['start_lng'].iloc[0]
lngnum

In [ ]:
dtnum = sep_dis_df['unix_dt'].iloc[i]
dtnum

In [ ]:
api_url = f"http://api.openweathermap.org/data/3.0/onecall/timemachine?lat={latnum}&lon={lngnum}&dt={dtnum}&units=imperial&appid={config.api_key}"
response = requests.get(api_url)
resp = response.json()

In [ ]:
resp